In [2]:
import os
import pandas as pd
import numpy as np
import datetime
from datetime import date

import re
import requests
from bs4 import BeautifulSoup as bs

import tiktoken

os.getcwd()

'c:\\Users\\Denis_Davydov2\\OneDrive\\Scipts\\Py_Scripts\\EPAM\\Prophet\\AI_skills'

# Get list of URLs

In [1]:
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import time
import logging

# Configure logging
logging.basicConfig(
    filename='scraping_errors.log',
    filemode='a',
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.ERROR
)

# Base URL
BASE_URL = "https://news.smol.ai/issues"

# Headers to mimic a browser request
HEADERS = {
    "User-Agent": "Chrome/134.0 (compatible; Web/1.0; +http://aipsychometrics.com/)"
}

def is_valid(url):
    """
    Check if the URL is valid, belongs to the desired domain and path,
    and does not contain a fragment identifier.
    """
    try:
        parsed = urlparse(url)
        return (
            parsed.scheme in ["http", "https"] and
            parsed.netloc.endswith("news.smol.ai") and
            parsed.path.startswith("/issues/") and
            not parsed.fragment  # Exclude URLs with fragments
        )
    except ValueError as e:
        logging.error(f"URL parsing failed for '{url}': {e}")
        return False

def get_all_links(url, collected_urls):
    """
    Recursively fetch and collect all valid URLs from the given URL.
    """
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        response.raise_for_status()  # Raise an exception for HTTP errors
    except requests.RequestException as e:
        logging.error(f"Failed to retrieve {url}: {e}")
        return

    soup = BeautifulSoup(response.text, "html.parser")

    for link in soup.find_all("a", href=True):
        href = link.get("href")
        # Ignore links that start with '#' to skip fragment-only URLs
        if href.startswith("#"):
            continue
        try:
            full_url = urljoin(BASE_URL, href)
        except Exception as e:
            logging.error(f"Error joining URL '{href}' with base '{BASE_URL}': {e}")
            continue
        # Remove any fragments from the URL
        full_url = full_url.split('#')[0]
        if is_valid(full_url):
            if full_url not in collected_urls:
                collected_urls.add(full_url)
                print(f"Found URL: {full_url}")
                # To prevent overwhelming the server, pause between requests
                time.sleep(0.3)
                get_all_links(full_url, collected_urls)

def main():
    collected_urls = set()
    print(f"Starting to scrape from: {BASE_URL}")
    get_all_links(BASE_URL, collected_urls)

    # Save the collected URLs to a file
    with open("smolai_urls.txt", "w") as f:
        for url in sorted(collected_urls):
            f.write(url + "\n")

    print(f"Scraping completed. {len(collected_urls)} URLs found.")
    print("URLs have been saved to ***_urls.txt")
    print("Any errors have been logged to scraping_errors.log")


    return pd.DataFrame(collected_urls)


if __name__ == "__main__":
    main()
    df_urls = main()

Starting to scrape from: https://news.smol.ai/issues
Found URL: https://news.smol.ai/issues/
Found URL: https://news.smol.ai/issues/25-10-28-openai-restructure
Found URL: https://news.smol.ai/issues/25-10-13-oai-broadcom
Found URL: https://news.smol.ai/issues/25-09-10-oci
Found URL: https://news.smol.ai/issues/25-09-11-qwen3-next
Found URL: https://news.smol.ai/issues/24-05-06-ainews-deepseek-v2-beats-mixtral-8x22b-with-greater160-experts-at-half-the-cost
Found URL: https://news.smol.ai/issues/24-05-07-ainews-kolmogorov-arnold-networks-mlp-killers-or-just-spicy-mlps
Found URL: https://news.smol.ai/issues/24-05-08-ainews-openais-pr-campaign
Found URL: https://news.smol.ai/issues/24-05-09-ainews-lmsys-advances-llama-3-eval-analysis
Found URL: https://news.smol.ai/issues/24-05-10-ainews-quis-promptum-ipso-promptiet
Found URL: https://news.smol.ai/issues/24-05-13-ainews-gpt-4o-the-new-sota-everything-frontier-model-gpt4o-version
Found URL: https://news.smol.ai/issues/24-05-13-ainews-gpt-4o

In [ ]:
# Ver2
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import time
import logging

# Configure logging
logging.basicConfig(
    filename='scraping_errors.log',
    filemode='a',
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.ERROR
)

# Base URL
BASE_URL = "https://buttondown.com/ainews/archive/"

# Headers to mimic a browser request
HEADERS = {
    "User-Agent": "Chrome/134.0 (compatible; Web/1.0; +http://aipsychometrics.com/)"
}

def is_valid(url):
    """
    Check if the URL is valid, belongs to the desired domain and path,
    and does not contain a fragment identifier.
    """
    try:
        parsed = urlparse(url)
        return (
            parsed.scheme in ["http", "https"] and
            parsed.netloc.endswith("buttondown.com") and
            parsed.path.startswith("/ainews/archive/ainews-") and  # Updated to only accept URLs starting with 'ainews-'
            not parsed.fragment  # Exclude URLs with fragments
        )
    except ValueError as e:
        logging.error(f"URL parsing failed for '{url}': {e}")
        return False

def get_all_links(url, collected_urls):
    """
    Recursively fetch and collect all valid URLs from the given URL.
    """
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        response.raise_for_status()  # Raise an exception for HTTP errors
    except requests.RequestException as e:
        logging.error(f"Failed to retrieve {url}: {e}")
        return

    soup = BeautifulSoup(response.text, "html.parser")

    for link in soup.find_all("a", href=True):
        href = link.get("href")
        # Ignore links that start with '#' to skip fragment-only URLs
        if href.startswith("#"):
            continue
        try:
            full_url = urljoin(BASE_URL, href)
        except Exception as e:
            logging.error(f"Error joining URL '{href}' with base '{BASE_URL}': {e}")
            continue
        # Remove any fragments from the URL
        full_url = full_url.split('#')[0]
        if is_valid(full_url):  # Check if the URL is valid
            if full_url not in collected_urls:
                collected_urls.add(full_url)
                print(f"Found URL: {full_url}")
                # To prevent overwhelming the server, pause between requests
                time.sleep(0.3)
                get_all_links(full_url, collected_urls)

def main():
    collected_urls = set()
    print(f"Starting to scrape from: {BASE_URL}")
    get_all_links(BASE_URL, collected_urls)

    # Save the collected URLs to a file
    with open("buttondown_archive_urls.txt", "w") as f:
        for url in sorted(collected_urls):
            f.write(url + "\n")

    print(f"Scraping completed. {len(collected_urls)} URLs found.")
    print("URLs have been saved to buttondown_archive_urls.txt")
    print("Any errors have been logged to scraping_errors.log")

    return pd.DataFrame(collected_urls)

if __name__ == "__main__":
    df_urls = main()  # Call main and store result
    print(df_urls)  # Display the DataFrame

Starting to scrape from: https://buttondown.com/ainews/archive/
Found URL: https://buttondown.com/ainews/archive/ainews-qwq-32b-claims-to-match-deepseek-r1-671b/
Found URL: https://buttondown.com/ainews/archive/ainews-not-much-happened-today-6597/
Found URL: https://buttondown.com/ainews/archive/ainews-not-much-happened-today-6801/
Found URL: https://buttondown.com/ainews/archive/ainews-not-much-happened-today-2943/
Found URL: https://buttondown.com/ainews/archive/ainews-41b-raised-today-openai-300b-cursor-95b/
Found URL: https://buttondown.com/ainews/archive/ainews-not-much-happened-today-9938/
Found URL: https://buttondown.com/ainews/archive/ainews-not-much-happened-today-3156/
Found URL: https://buttondown.com/ainews/archive/ainews-ghibli-memes/
Found URL: https://buttondown.com/ainews/archive/ainews-gemini-25-pro-4o-native-image-gen/
Found URL: https://buttondown.com/ainews/archive/ainews-xai-grok-3-and-mira-muratis-thinking/
Found URL: https://buttondown.com/ainews/archive/ainews-

# Get full texts

In [5]:
file_path = 'C:\\Users\\Denis_Davydov2\\OneDrive\\Scipts\\Py_Scripts\\EPAM\\Prophet\\AI_skills\\'

print(file_path)
file_name = 'smolai_urls.txt'
print("File read: ", file_name)

links = pd.read_csv(file_path+file_name, header=None)
print("Toral downloaded links:", len(links))

links['urls'] = links[0].astype(str)
links

C:\Users\Denis_Davydov2\OneDrive\Scipts\Py_Scripts\EPAM\Prophet\AI_skills\
File read:  smolai_urls.txt
Toral downloaded links: 497


,0,urls
0,https://news.smol.ai/issues/,https://news.smol.ai/issues/
1,https://news.smol.ai/issues/23-12-06-ainews-is...,https://news.smol.ai/issues/23-12-06-ainews-is...
2,https://news.smol.ai/issues/23-12-07-ainews-12...,https://news.smol.ai/issues/23-12-07-ainews-12...
3,https://news.smol.ai/issues/23-12-08-ainews-12...,https://news.smol.ai/issues/23-12-08-ainews-12...
4,https://news.smol.ai/issues/23-12-09-ainews-12...,https://news.smol.ai/issues/23-12-09-ainews-12...
...,...,...
492,https://news.smol.ai/issues/25-10-23-not-much,https://news.smol.ai/issues/25-10-23-not-much
493,https://news.smol.ai/issues/25-10-24-not-much,https://news.smol.ai/issues/25-10-24-not-much
494,https://news.smol.ai/issues/25-10-27-minimax-m2,https://news.smol.ai/issues/25-10-27-minimax-m2
495,https://news.smol.ai/issues/25-10-28-openai-re...,https://news.smol.ai/issues/25-10-28-openai-re...


In [6]:
import io

links['date'] = links['urls'].str.extract(r'(\d{2}-\d{2}-\d{2})')


links.dropna(subset=['date'], inplace=True)


links['date'] = pd.to_datetime(links['date'], format='%y-%m-%d')
print(links.dtypes)

links = links.sort_values(by='date')
links = links[links['date'] >= '2024-01-01']


print("\n----------- Final DataFrame (Sorted and Filtered) -----------")
links


0               object
urls            object
date    datetime64[ns]
dtype: object

----------- 5. Final DataFrame (Sorted and Filtered) -----------


,0,urls,date
26,https://news.smol.ai/issues/24-01-02-ainews-11...,https://news.smol.ai/issues/24-01-02-ainews-11...,2024-01-02
27,https://news.smol.ai/issues/24-01-02-ainews-12...,https://news.smol.ai/issues/24-01-02-ainews-12...,2024-01-02
28,https://news.smol.ai/issues/24-01-03-ainews-13...,https://news.smol.ai/issues/24-01-03-ainews-13...,2024-01-03
29,https://news.smol.ai/issues/24-01-05-ainews-14...,https://news.smol.ai/issues/24-01-05-ainews-14...,2024-01-05
30,https://news.smol.ai/issues/24-01-07-ainews-16...,https://news.smol.ai/issues/24-01-07-ainews-16...,2024-01-07
...,...,...,...
491,https://news.smol.ai/issues/25-10-22-not-much,https://news.smol.ai/issues/25-10-22-not-much,2025-10-22
492,https://news.smol.ai/issues/25-10-23-not-much,https://news.smol.ai/issues/25-10-23-not-much,2025-10-23
493,https://news.smol.ai/issues/25-10-24-not-much,https://news.smol.ai/issues/25-10-24-not-much,2025-10-24
494,https://news.smol.ai/issues/25-10-27-minimax-m2,https://news.smol.ai/issues/25-10-27-minimax-m2,2025-10-27


In [7]:
urls = links['urls'].tolist()
urls

['https://news.smol.ai/issues/24-01-02-ainews-112024-how-to-start-with-open-source-ai',
 'https://news.smol.ai/issues/24-01-02-ainews-122024-smol-tweaks-to-smol-talk',
 'https://news.smol.ai/issues/24-01-03-ainews-132024-rip-coqui',
 'https://news.smol.ai/issues/24-01-05-ainews-142024-jeff-bezos-backs-perplexitys-dollar520m-series-b',
 'https://news.smol.ai/issues/24-01-07-ainews-16-72024-llama-pro-an-alternative-to-peftrag',
 'https://news.smol.ai/issues/24-01-08-ainews-182024-the-four-wars-of-the-ai-stack',
 'https://news.smol.ai/issues/24-01-10-ainews-192024-nous-research-lands-dollar5m-for-open-source-ai',
 'https://news.smol.ai/issues/24-01-11-ainews-1102024-all-the-best-papers-for-ai-engineers',
 'https://news.smol.ai/issues/24-01-12-ainews-1112024-mixing-experts-vs-merging-models',
 'https://news.smol.ai/issues/24-01-13-ainews-1122024-anthropic-coins-sleeper-agents',
 'https://news.smol.ai/issues/24-01-15-ainews-113-142024-dont-sleep-on-prompt-engineering',
 'https://news.smol.a

## Save list of urls

In [19]:
file_path = r'C:\Users\Denis_Davydov2\OneDrive - EPAM\Prophet_AI_docs\Datasets\AI_skills'
file_name = f'urls.txt'

with open(file_path + '\\' + file_name, "w") as output:
    output.write(str(urls))
    
print(f"File {file_name} \nuploaded to:", file_path)


File urls.txt 
uploaded to: C:\Users\Denis_Davydov2\OneDrive - EPAM\Prophet_AI_docs\Datasets\AI_skills


## Extract text from site

In [10]:
urls = ['https://news.smol.ai/issues/25-06-03-not-much',
 'https://news.smol.ai/issues/25-06-04-not-much',
 'https://news.smol.ai/issues/25-06-05-aia']

In [36]:
# Get list of URLs from file

file_path = r'C:\Users\Denis_Davydov2\OneDrive - EPAM\Prophet_AI_docs\Datasets\AI_skills\urls.txt'

with open(file_path, 'r') as file:
    content = file.read()
    # Convert string representation of list back to list
    urls = eval(content)

print(f"Number of URLs loaded: {len(urls)}")
urls

Number of URLs loaded: 407


['https://news.smol.ai/issues/24-04-01-ainews-adamw-greater-aarond',
 'https://news.smol.ai/issues/24-04-02-ainews-not-much-happened-today',
 'https://news.smol.ai/issues/24-04-03-ainews-realm-reference-resolution-as-language-modeling',
 'https://news.smol.ai/issues/24-04-04-ainews-cohere-command-r-anthropic-claude-tool-use-openai-finetuning',
 'https://news.smol.ai/issues/24-04-05-ainews-mixture-of-depths-dynamically-allocating-compute-in-transformer-based-language-models',
 'https://news.smol.ai/issues/24-04-08-ainews-anime-pfp-anon-eclipses-dollar10k-ab-prompting-challenge',
 'https://news.smol.ai/issues/24-04-09-ainews-gemini-pro-and-gpt4t-vision-go-ga-on-the-same-day-by-complete-coincidence',
 'https://news.smol.ai/issues/24-04-10-ainews-musics-dall-e-moment',
 'https://news.smol.ai/issues/24-04-11-ainews-mergestral-meta-mtiav2-cohere-rerank-3-google-infini-attention',
 'https://news.smol.ai/issues/24-04-12-ainews-zero-to-gpt-in-1-year',
 'https://news.smol.ai/issues/24-04-15-aine

In [30]:
# Short samples of links
urls = urls[:3]
urls

['https://news.smol.ai/issues/24-04-01-ainews-adamw-greater-aarond',
 'https://news.smol.ai/issues/24-04-02-ainews-not-much-happened-today',
 'https://news.smol.ai/issues/24-04-03-ainews-realm-reference-resolution-as-language-modeling']

In [37]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import lxml
import datetime

def fetch_html(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (compatible; WebScraper/1.0; +http://effcon.com/)"
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
        return None

def extract_section_by_id(soup, section_id):
    """Extract text from a section identified by its h1 id."""
    # Find the h1 with the specific id
    section_header = soup.find('h1', {'id': section_id})
    
    if not section_header:
        return None
    
    section_content = []
    
    # Get all siblings after the header until we hit another h1 or hr
    for sibling in section_header.find_next_siblings():
        # Stop if we hit another major section
        if sibling.name == 'h1' or sibling.name == 'hr':
            break
        
        # Collect text from relevant tags
        if sibling.name in ['p', 'ul', 'ol', 'h2', 'h3', 'h4', 'blockquote']:
            text = sibling.get_text(separator=' ', strip=True)
            if text:
                section_content.append(text)
    
    return '\n'.join(section_content) if section_content else None

def extract_data(url):
    """Extracts data from a single URL."""
    html = fetch_html(url)
    if not html:
        return None
    
    soup = BeautifulSoup(html, 'lxml')
    
    # Extract title from the <title> tag or h1
    title = None
    title_tag = soup.find('title')
    if title_tag:
        title_text = title_tag.get_text(strip=True)
        # Remove " | AINews" suffix and clean up
        title = title_text.replace(' | AINews', '').strip()
    
    # Extract date from meta tag or content
    date_text = None
    # Try meta tag first
    meta_date = soup.find('meta', {'property': 'article:published_time'})
    if meta_date and meta_date.get('content'):
        date_str = meta_date.get('content')
        # Parse and format as "Oct 22"
        try:
            from datetime import datetime as dt
            date_obj = dt.fromisoformat(date_str.replace('Z', '+00:00'))
            date_text = date_obj.strftime('%b %d')
        except:
            pass
    
    # Extract news period from main content
    news_period = None
    main_content = soup.find('article') or soup.find('main')
    if main_content:
        period_pattern = r'\b(\d{1,2}/\d{1,2}/\d{4}-\d{1,2}/\d{1,2}/\d{4})\b'
        period_match = re.search(period_pattern, main_content.get_text())
        if period_match:
            news_period = period_match.group(1)
    
    # Extract specific recap sections by their IDs
    twitter_text = extract_section_by_id(soup, 'ai-twitter-recap')
    reddit_text = extract_section_by_id(soup, 'ai-reddit-recap')
    discord_text = extract_section_by_id(soup, 'ai-discord-recap')

    return {
        'url': url,
        'title': title,
        'date': date_text,
        'news_period': news_period,
        'twitter_recap': twitter_text,
        'reddit_recap': reddit_text,
        'discord_recap': discord_text
    }

def main(urls):
    """Processes multiple URLs and returns a DataFrame."""
    all_data = []
    for url in urls:
        print(f"Processing URL: {url}")
        data = extract_data(url)
        if data:  # Append if data extraction is successful
            all_data.append(data)
    return pd.DataFrame(all_data)

# Execution
now1 = datetime.datetime.now()
print("start: ", now1)

if __name__ == "__main__":
    raw_text = main(urls)
    display(raw_text)

now2 = datetime.datetime.now()
print("finish: ", now2)
print(now2-now1)

start:  2025-10-30 13:51:09.266146
Processing URL: https://news.smol.ai/issues/24-04-01-ainews-adamw-greater-aarond
Processing URL: https://news.smol.ai/issues/24-04-02-ainews-not-much-happened-today
Processing URL: https://news.smol.ai/issues/24-04-02-ainews-not-much-happened-today
Processing URL: https://news.smol.ai/issues/24-04-03-ainews-realm-reference-resolution-as-language-modeling
Processing URL: https://news.smol.ai/issues/24-04-03-ainews-realm-reference-resolution-as-language-modeling
Processing URL: https://news.smol.ai/issues/24-04-04-ainews-cohere-command-r-anthropic-claude-tool-use-openai-finetuning
Processing URL: https://news.smol.ai/issues/24-04-04-ainews-cohere-command-r-anthropic-claude-tool-use-openai-finetuning
Processing URL: https://news.smol.ai/issues/24-04-05-ainews-mixture-of-depths-dynamically-allocating-compute-in-transformer-based-language-models
Processing URL: https://news.smol.ai/issues/24-04-05-ainews-mixture-of-depths-dynamically-allocating-compute-in-

,url,title,date,news_period,twitter_recap,reddit_recap,discord_recap
0,https://news.smol.ai/issues/24-04-01-ainews-ad...,AdamW -> AaronD?,Apr 01,3/28/2024-4/1/2024,"all recaps done by Claude 3 Opus, best of 4 ru...","Across r/LocalLlama, r/machinelearning, r/open...",None
1,https://news.smol.ai/issues/24-04-02-ainews-no...,Not much happened today,Apr 02,4/1/2024-4/2/2024,"all recaps done by Claude 3 Opus, best of 4 ru...","Across r/LocalLlama, r/machinelearning, r/open...",A summary of Summaries of Summaries\nClaude 3 ...
2,https://news.smol.ai/issues/24-04-03-ainews-re...,ReALM: Reference Resolution As Language Modeling,Apr 04,4/2/2024-4/3/2024,"all recaps done by Claude 3 Opus, best of 4 ru...","Across r/LocalLlama, r/machinelearning, r/open...",A summary of Summaries of Summaries\nAdvanceme...
3,https://news.smol.ai/issues/24-04-04-ainews-co...,"Cohere Command R+, Anthropic Claude Tool Use, ...",Apr 04,4/3/2024-4/4/2024,"all recaps done by Claude 3 Opus, best of 4 ru...","Across r/LocalLlama, r/machinelearning, r/open...",A summary of Summaries of Summaries\nLLM Advan...
4,https://news.smol.ai/issues/24-04-05-ainews-mi...,Mixture of Depths: Dynamically allocating comp...,Apr 05,4/4/2024-4/5/2024,"all recaps done by Claude 3 Opus, best of 4 ru...","Across r/LocalLlama, r/machinelearning, r/open...",A summary of Summaries of Summaries\n1. Cuttin...
...,...,...,...,...,...,...,...
402,https://news.smol.ai/issues/25-10-23-not-much,not much happened today,Oct 23,10/22/2025-10/23/2025,"Agent ops, observability, and real-world envs\...",/r/LocalLlama + /r/localLLM Recap\n1. AI Agent...,A summary of Summaries of Summaries by X.ai Gr...
403,https://news.smol.ai/issues/25-10-24-not-much,not much happened today,Oct 24,10/23/2025-10/24/2025,Serving and Production Platforms: vLLM x NVIDI...,/r/LocalLlama + /r/localLLM Recap\nGLM-4.6-Air...,A summary of Summaries of Summaries by X.ai Gr...
404,https://news.smol.ai/issues/25-10-27-minimax-m2,MiniMax M2 230BA10B — 8% of Claude Sonnet's pr...,Oct 27,10/24/2025-10/27/2025,MiniMax M2 open-weights release: sparse MoE fo...,/r/LocalLlama + /r/localLLM Recap\n1. Open-Sou...,A summary of Summaries of Summaries by Gemini ...
405,https://news.smol.ai/issues/25-10-28-openai-re...,OpenAI completes Microsoft + For-profit restru...,Oct 28,10/27/2025-10/28/2025,"OpenAI’s new structure, Microsoft deal, and “o...",/r/LocalLlama + /r/localLLM Recap\n1. DGX Spar...,A summary of Summaries of Summaries by gpt-5\n...


finish:  2025-10-30 13:58:23.522501
0:07:14.256355


### Check result output

In [81]:
print("Full content of the row:\n")
for column in raw_text.columns:
    print(f"{column}:")
    print(raw_text.iloc[0][column])
    print("-" * 80)

Full content of the row:

url:
https://news.smol.ai/issues/24-04-01-ainews-adamw-greater-aarond
--------------------------------------------------------------------------------
title:
AdamW -> AaronD?
--------------------------------------------------------------------------------
date:
2024-04-01 00:00:00
--------------------------------------------------------------------------------
news_period:
3/28/2024-4/1/2024
--------------------------------------------------------------------------------
twitter_recap:
all recaps done by Claude 3 Opus, best of 4 runs. We are working on clustering and flow engineering with Haiku.
AI Capabilities and Limitations
Limitations of current AI systems : @fchollet noted that memorization, which ML has solely focused on, is not intelligence. Any task that does not involve significant novelty and uncertainty can be solved via memorization, but skill is never a sign of intelligence. @fchollet shared a paper introducing a formal definition of intelligence 

In [40]:

# Print the 'twitter_recap' text from the first row of raw_text
if 'twitter_recap' in raw_text.columns and not raw_text.empty:
    first_idx = raw_text.index[300]
    print(raw_text.at[first_idx, 'twitter_recap'])
else:
    print("Column 'twitter_recap' missing or raw_text is empty")

Here’s the breakdown of tweets, categorized and summarized as requested:
Language Models, Architectures, and Implementations
Ideal architecture for inference : @tri_dao discusses the need for an “ideal” architecture in the era of inference-driven AI, highlighting attention variants like GTA & GLA, tailored for high arithmetic intensity , easy sharding for big models, and high model quality. @tri_dao also notes insights from the project, such as cutting KV cache size in half by sharing K & V, increasing arithmetic intensity. Their GTA leverages decoupled RoPE to retain the quality of GQA at half the KV cache size. DeepSeek MLA and Arithmetic Intensity : @tri_dao highlights that Deepseek MLA is the first attention variant that can hit a compute-bound regime during inference decode due to its high arithmetic intensity (~256) . @tri_dao suggests that GTA is an easy replacement for GQA, and GLA is a good replacement for MLA. LayerNorm kernel replication : @fleetwood___ replicated the LayerN

## Save raw_text to CSV

In [42]:
file_path = r'C:\Users\Denis_Davydov2\OneDrive - EPAM\Prophet_AI_docs\Datasets\AI_skills'
file_name = f'smolai_raw_{date.today()}.csv'

raw_text.to_csv(file_path + '\\' + file_name, index=False, encoding='utf-8-sig')
print(f"File {file_name} \nuploaded to:", file_path)
print("Total rows", len(raw_text))

File smolai_raw_2025-10-30.csv 
uploaded to: C:\Users\Denis_Davydov2\OneDrive - EPAM\Prophet_AI_docs\Datasets\AI_skills
Total rows 407


## Text cleaning and preparation

### Get saved data 

In [53]:
# Get data Inoreader 2025
file_path = 'C:\\Users\\Denis_Davydov2\\OneDrive - EPAM\\Prophet_AI_docs\\Datasets\\AI_skills\\'
file_name = 'smolai_raw_2025-10-30.csv'

try:
    smalai = pd.read_csv(file_path+file_name, encoding='utf-8-sig')
    print("File read successfully with utf-8-sig encoding")
except UnicodeDecodeError:
    try:
        smalai = pd.read_csv(file_path+file_name, encoding='latin-1')
        print("File read successfully with latin-1 encoding")
    except:
        smalai = pd.read_csv(file_path+file_name, encoding='cp1252')
        print("File read successfully with cp1252 encoding")

print("File read: ", file_name)
smalai

File read successfully with latin-1 encoding
File read:  smolai_raw_2025-10-30.csv


,url,title,date,news_period,twitter_recap,reddit_recap,discord_recap
0,https://news.smol.ai/issues/25-09-10-oci,Oracle jumps +36% in a day after winning $300B...,10-Sep,9/9/2025-9/10/2025,Fast RL for Tool-Use and Weight Update Infrast...,/r/LocalLlama + /r/localLLM Recap\n1. Unsloth ...,Theme 1: Models Muscle Up with Speedy Tweaks\n...
1,https://news.smol.ai/issues/24-09-10-ainews-no...,not much happened today + AINews Podcast?,11-Sep,9/9/2024-9/10/2024,\nAppleâs AI Announcements and Industry Reac...,/r/LocalLlama Recap\nTheme 1. Reflection 70B: ...,A summary of Summaries of Summaries by Claude ...
2,https://news.smol.ai/issues/25-09-09-not-much,not much happened today,9-Sep,9/8/2025-9/9/2025,Coding Agents and Tooling Momentum\nCognition ...,/r/LocalLlama + /r/localLLM Recap\n1. A3B HF R...,NaN
3,https://news.smol.ai/issues/24-09-09-ainews-ai...,AIPhone 16: the Visual Intelligence Phone,9-Sep,9/6/2024-9/9/2024,\nAI Model Developments and Benchmarks\nReflec...,/r/LocalLlama Recap\nTheme 1. Reflection 70B C...,A summary of Summaries of Summaries GPT4O (gpt...
4,https://news.smol.ai/issues/25-09-08-cog-smol,Cognition's $10b Series C; Smol AI updates,8-Sep,9/5/2025-9/8/2025,Coding Agents and Tooling Momentum\nCognition ...,/r/LocalLlama + /r/localLLM Recap\n1. Open-sou...,NaN
...,...,...,...,...,...,...,...
402,https://news.smol.ai/issues/25-01-16-ainews-no...,not much happened today,17-Jan,1/15/2025-1/16/2025,\nAI Model Developments\nAdvanced Text-to-Spee...,/r/LocalLlama Recap\nTheme 1. Googleâs Neura...,A summary of Summaries of Summaries by o1-prev...
403,https://news.smol.ai/issues/25-01-15-ainews-ti...,Titans: Learning to Memorize at Test Time,16-Jan,1/14/2025-1/15/2025,\nAI Models and Scaling\nMiniMax-01 and Ultra-...,/r/LocalLlama Recap\nTheme 1. InternLM3-8B out...,A summary of Summaries of Summaries by o1-prev...
404,https://news.smol.ai/issues/25-01-14-ainews-sm...,small little news items,15-Jan,1/13/2025-1/14/2025,\nModel Releases and Updates\nOllama Model Enh...,/r/LocalLlama Recap\nTheme 1. Qwenâs Math Pr...,A summary of Summaries of Summaries by o1-prev...
405,https://news.smol.ai/issues/25-01-13-ainews-no...,not much happened today,14-Jan,1/10/2025-1/13/2025,\nAI Model Releases & Benchmarks\nHelium-1 Pre...,/r/LocalLlama Recap\nTheme 1. Criticism of â...,A summary of Summaries of Summaries by o1-2024...


### Сontinue with previous dataframe

In [99]:
raw_text

,url,title,date,news_period,twitter_recap,reddit_recap,discord_recap,content,date_extracted
0,https://news.smol.ai/issues/24-04-01-ainews-ad...,AdamW -> AaronD?,2024-04-01,3/28/2024-4/1/2024,"all recaps done by Claude 3 Opus, best of 4 ru...","Across r/LocalLlama, r/machinelearning, r/open...",None,NaN,2024-04-01
1,https://news.smol.ai/issues/24-04-02-ainews-no...,Not much happened today,2024-04-02,4/1/2024-4/2/2024,"all recaps done by Claude 3 Opus, best of 4 ru...","Across r/LocalLlama, r/machinelearning, r/open...",A summary of Summaries of Summaries\nClaude 3 ...,"all recaps done by Claude 3 Opus, best of 4 ru...",2024-04-02
2,https://news.smol.ai/issues/24-04-03-ainews-re...,ReALM: Reference Resolution As Language Modeling,2024-04-03,4/2/2024-4/3/2024,"all recaps done by Claude 3 Opus, best of 4 ru...","Across r/LocalLlama, r/machinelearning, r/open...",A summary of Summaries of Summaries\nAdvanceme...,"all recaps done by Claude 3 Opus, best of 4 ru...",2024-04-03
3,https://news.smol.ai/issues/24-04-04-ainews-co...,"Cohere Command R+, Anthropic Claude Tool Use, ...",2024-04-04,4/3/2024-4/4/2024,"all recaps done by Claude 3 Opus, best of 4 ru...","Across r/LocalLlama, r/machinelearning, r/open...",A summary of Summaries of Summaries\nLLM Advan...,"all recaps done by Claude 3 Opus, best of 4 ru...",2024-04-04
4,https://news.smol.ai/issues/24-04-05-ainews-mi...,Mixture of Depths: Dynamically allocating comp...,2024-04-05,4/4/2024-4/5/2024,"all recaps done by Claude 3 Opus, best of 4 ru...","Across r/LocalLlama, r/machinelearning, r/open...",A summary of Summaries of Summaries\n1. Cuttin...,"all recaps done by Claude 3 Opus, best of 4 ru...",2024-04-05
...,...,...,...,...,...,...,...,...,...
402,https://news.smol.ai/issues/25-10-23-not-much,not much happened today,2025-10-23,10/22/2025-10/23/2025,"Agent ops, observability, and real-world envs\...",/r/LocalLlama + /r/localLLM Recap\n1. AI Agent...,A summary of Summaries of Summaries by X.ai Gr...,"Agent ops, observability, and real-world envs\...",2025-10-23
403,https://news.smol.ai/issues/25-10-24-not-much,not much happened today,2025-10-24,10/23/2025-10/24/2025,Serving and Production Platforms: vLLM x NVIDI...,/r/LocalLlama + /r/localLLM Recap\nGLM-4.6-Air...,A summary of Summaries of Summaries by X.ai Gr...,Serving and Production Platforms: vLLM x NVIDI...,2025-10-24
404,https://news.smol.ai/issues/25-10-27-minimax-m2,MiniMax M2 230BA10B — 8% of Claude Sonnet's pr...,2025-10-27,10/24/2025-10/27/2025,MiniMax M2 open-weights release: sparse MoE fo...,/r/LocalLlama + /r/localLLM Recap\n1. Open-Sou...,A summary of Summaries of Summaries by Gemini ...,MiniMax M2 open-weights release: sparse MoE fo...,2025-10-27
405,https://news.smol.ai/issues/25-10-28-openai-re...,OpenAI completes Microsoft + For-profit restru...,2025-10-28,10/27/2025-10/28/2025,"OpenAI’s new structure, Microsoft deal, and “o...",/r/LocalLlama + /r/localLLM Recap\n1. DGX Spar...,A summary of Summaries of Summaries by gpt-5\n...,"OpenAI’s new structure, Microsoft deal, and “o...",2025-10-28


In [87]:
raw_text.dropna(subset=['date', 'title'], inplace=True)
raw_text['date_extracted'] = raw_text['news_period'].str.extract(r'-(\d{1,2}/\d{1,2}/\d{4})')[0]
raw_text['date_extracted'] = pd.to_datetime(raw_text['date_extracted'], format='%m/%d/%Y')
raw_text.dropna(subset=['date', 'title'], inplace=True)
raw_text

,url,title,date,news_period,twitter_recap,reddit_recap,discord_recap,content,date_extracted
0,https://news.smol.ai/issues/24-04-01-ainews-ad...,AdamW -> AaronD?,2024-04-01,3/28/2024-4/1/2024,"all recaps done by Claude 3 Opus, best of 4 ru...","Across r/LocalLlama, r/machinelearning, r/open...",None,NaN,2024-04-01
1,https://news.smol.ai/issues/24-04-02-ainews-no...,Not much happened today,2024-04-02,4/1/2024-4/2/2024,"all recaps done by Claude 3 Opus, best of 4 ru...","Across r/LocalLlama, r/machinelearning, r/open...",A summary of Summaries of Summaries\nClaude 3 ...,"all recaps done by Claude 3 Opus, best of 4 ru...",2024-04-02
2,https://news.smol.ai/issues/24-04-03-ainews-re...,ReALM: Reference Resolution As Language Modeling,2024-04-03,4/2/2024-4/3/2024,"all recaps done by Claude 3 Opus, best of 4 ru...","Across r/LocalLlama, r/machinelearning, r/open...",A summary of Summaries of Summaries\nAdvanceme...,"all recaps done by Claude 3 Opus, best of 4 ru...",2024-04-03
3,https://news.smol.ai/issues/24-04-04-ainews-co...,"Cohere Command R+, Anthropic Claude Tool Use, ...",2024-04-04,4/3/2024-4/4/2024,"all recaps done by Claude 3 Opus, best of 4 ru...","Across r/LocalLlama, r/machinelearning, r/open...",A summary of Summaries of Summaries\nLLM Advan...,"all recaps done by Claude 3 Opus, best of 4 ru...",2024-04-04
4,https://news.smol.ai/issues/24-04-05-ainews-mi...,Mixture of Depths: Dynamically allocating comp...,2024-04-05,4/4/2024-4/5/2024,"all recaps done by Claude 3 Opus, best of 4 ru...","Across r/LocalLlama, r/machinelearning, r/open...",A summary of Summaries of Summaries\n1. Cuttin...,"all recaps done by Claude 3 Opus, best of 4 ru...",2024-04-05
...,...,...,...,...,...,...,...,...,...
402,https://news.smol.ai/issues/25-10-23-not-much,not much happened today,2025-10-23,10/22/2025-10/23/2025,"Agent ops, observability, and real-world envs\...",/r/LocalLlama + /r/localLLM Recap\n1. AI Agent...,A summary of Summaries of Summaries by X.ai Gr...,"Agent ops, observability, and real-world envs\...",2025-10-23
403,https://news.smol.ai/issues/25-10-24-not-much,not much happened today,2025-10-24,10/23/2025-10/24/2025,Serving and Production Platforms: vLLM x NVIDI...,/r/LocalLlama + /r/localLLM Recap\nGLM-4.6-Air...,A summary of Summaries of Summaries by X.ai Gr...,Serving and Production Platforms: vLLM x NVIDI...,2025-10-24
404,https://news.smol.ai/issues/25-10-27-minimax-m2,MiniMax M2 230BA10B — 8% of Claude Sonnet's pr...,2025-10-27,10/24/2025-10/27/2025,MiniMax M2 open-weights release: sparse MoE fo...,/r/LocalLlama + /r/localLLM Recap\n1. Open-Sou...,A summary of Summaries of Summaries by Gemini ...,MiniMax M2 open-weights release: sparse MoE fo...,2025-10-27
405,https://news.smol.ai/issues/25-10-28-openai-re...,OpenAI completes Microsoft + For-profit restru...,2025-10-28,10/27/2025-10/28/2025,"OpenAI’s new structure, Microsoft deal, and “o...",/r/LocalLlama + /r/localLLM Recap\n1. DGX Spar...,A summary of Summaries of Summaries by gpt-5\n...,"OpenAI’s new structure, Microsoft deal, and “o...",2025-10-28


In [128]:
# Merge 3 columns into one
smalai = raw_text.copy()
smalai['content'] = (
    smalai['twitter_recap']
        .str.cat([smalai['reddit_recap'], smalai['discord_recap']], sep='\n\n', na_rep='')
        .str.replace(r'(\n\n)+$', '', regex=True)  # trim trailing separators
        .str.strip()
)

smalai

,url,title,date,news_period,twitter_recap,reddit_recap,discord_recap,content,date_extracted
0,https://news.smol.ai/issues/24-04-01-ainews-ad...,AdamW -> AaronD?,2024-04-01,3/28/2024-4/1/2024,"all recaps done by Claude 3 Opus, best of 4 ru...","Across r/LocalLlama, r/machinelearning, r/open...",None,"all recaps done by Claude 3 Opus, best of 4 ru...",2024-04-01
1,https://news.smol.ai/issues/24-04-02-ainews-no...,Not much happened today,2024-04-02,4/1/2024-4/2/2024,"all recaps done by Claude 3 Opus, best of 4 ru...","Across r/LocalLlama, r/machinelearning, r/open...",A summary of Summaries of Summaries\nClaude 3 ...,"all recaps done by Claude 3 Opus, best of 4 ru...",2024-04-02
2,https://news.smol.ai/issues/24-04-03-ainews-re...,ReALM: Reference Resolution As Language Modeling,2024-04-03,4/2/2024-4/3/2024,"all recaps done by Claude 3 Opus, best of 4 ru...","Across r/LocalLlama, r/machinelearning, r/open...",A summary of Summaries of Summaries\nAdvanceme...,"all recaps done by Claude 3 Opus, best of 4 ru...",2024-04-03
3,https://news.smol.ai/issues/24-04-04-ainews-co...,"Cohere Command R+, Anthropic Claude Tool Use, ...",2024-04-04,4/3/2024-4/4/2024,"all recaps done by Claude 3 Opus, best of 4 ru...","Across r/LocalLlama, r/machinelearning, r/open...",A summary of Summaries of Summaries\nLLM Advan...,"all recaps done by Claude 3 Opus, best of 4 ru...",2024-04-04
4,https://news.smol.ai/issues/24-04-05-ainews-mi...,Mixture of Depths: Dynamically allocating comp...,2024-04-05,4/4/2024-4/5/2024,"all recaps done by Claude 3 Opus, best of 4 ru...","Across r/LocalLlama, r/machinelearning, r/open...",A summary of Summaries of Summaries\n1. Cuttin...,"all recaps done by Claude 3 Opus, best of 4 ru...",2024-04-05
...,...,...,...,...,...,...,...,...,...
402,https://news.smol.ai/issues/25-10-23-not-much,not much happened today,2025-10-23,10/22/2025-10/23/2025,"Agent ops, observability, and real-world envs\...",/r/LocalLlama + /r/localLLM Recap\n1. AI Agent...,A summary of Summaries of Summaries by X.ai Gr...,"Agent ops, observability, and real-world envs\...",2025-10-23
403,https://news.smol.ai/issues/25-10-24-not-much,not much happened today,2025-10-24,10/23/2025-10/24/2025,Serving and Production Platforms: vLLM x NVIDI...,/r/LocalLlama + /r/localLLM Recap\nGLM-4.6-Air...,A summary of Summaries of Summaries by X.ai Gr...,Serving and Production Platforms: vLLM x NVIDI...,2025-10-24
404,https://news.smol.ai/issues/25-10-27-minimax-m2,MiniMax M2 230BA10B — 8% of Claude Sonnet's pr...,2025-10-27,10/24/2025-10/27/2025,MiniMax M2 open-weights release: sparse MoE fo...,/r/LocalLlama + /r/localLLM Recap\n1. Open-Sou...,A summary of Summaries of Summaries by Gemini ...,MiniMax M2 open-weights release: sparse MoE fo...,2025-10-27
405,https://news.smol.ai/issues/25-10-28-openai-re...,OpenAI completes Microsoft + For-profit restru...,2025-10-28,10/27/2025-10/28/2025,"OpenAI’s new structure, Microsoft deal, and “o...",/r/LocalLlama + /r/localLLM Recap\n1. DGX Spar...,A summary of Summaries of Summaries by gpt-5\n...,"OpenAI’s new structure, Microsoft deal, and “o...",2025-10-28


#### Cleaning

In [129]:
from html import unescape

def clean_text(text):
    if pd.isna(text):
        return pd.NA
    s = str(text)
    # remove any HTML tags
    s = re.sub(r'<[^>]+>', ' ', s)
    # unescape HTML entities
    s = unescape(s)
    # remove emojis and other non-ASCII symbols (keeps only ASCII printable chars)
    s = ''.join(ch for ch in s if ord(ch) < 128 and ch.isprintable())
    # normalize whitespace
    s = re.sub(r'\s+', ' ', s).strip()
    return s if s else pd.NA

smalai['content_cleaned'] = smalai['content'].apply(clean_text)
smalai

,url,title,date,news_period,twitter_recap,reddit_recap,discord_recap,content,date_extracted,content_cleaned
0,https://news.smol.ai/issues/24-04-01-ainews-ad...,AdamW -> AaronD?,2024-04-01,3/28/2024-4/1/2024,"all recaps done by Claude 3 Opus, best of 4 ru...","Across r/LocalLlama, r/machinelearning, r/open...",None,"all recaps done by Claude 3 Opus, best of 4 ru...",2024-04-01,"all recaps done by Claude 3 Opus, best of 4 ru..."
1,https://news.smol.ai/issues/24-04-02-ainews-no...,Not much happened today,2024-04-02,4/1/2024-4/2/2024,"all recaps done by Claude 3 Opus, best of 4 ru...","Across r/LocalLlama, r/machinelearning, r/open...",A summary of Summaries of Summaries\nClaude 3 ...,"all recaps done by Claude 3 Opus, best of 4 ru...",2024-04-02,"all recaps done by Claude 3 Opus, best of 4 ru..."
2,https://news.smol.ai/issues/24-04-03-ainews-re...,ReALM: Reference Resolution As Language Modeling,2024-04-03,4/2/2024-4/3/2024,"all recaps done by Claude 3 Opus, best of 4 ru...","Across r/LocalLlama, r/machinelearning, r/open...",A summary of Summaries of Summaries\nAdvanceme...,"all recaps done by Claude 3 Opus, best of 4 ru...",2024-04-03,"all recaps done by Claude 3 Opus, best of 4 ru..."
3,https://news.smol.ai/issues/24-04-04-ainews-co...,"Cohere Command R+, Anthropic Claude Tool Use, ...",2024-04-04,4/3/2024-4/4/2024,"all recaps done by Claude 3 Opus, best of 4 ru...","Across r/LocalLlama, r/machinelearning, r/open...",A summary of Summaries of Summaries\nLLM Advan...,"all recaps done by Claude 3 Opus, best of 4 ru...",2024-04-04,"all recaps done by Claude 3 Opus, best of 4 ru..."
4,https://news.smol.ai/issues/24-04-05-ainews-mi...,Mixture of Depths: Dynamically allocating comp...,2024-04-05,4/4/2024-4/5/2024,"all recaps done by Claude 3 Opus, best of 4 ru...","Across r/LocalLlama, r/machinelearning, r/open...",A summary of Summaries of Summaries\n1. Cuttin...,"all recaps done by Claude 3 Opus, best of 4 ru...",2024-04-05,"all recaps done by Claude 3 Opus, best of 4 ru..."
...,...,...,...,...,...,...,...,...,...,...
402,https://news.smol.ai/issues/25-10-23-not-much,not much happened today,2025-10-23,10/22/2025-10/23/2025,"Agent ops, observability, and real-world envs\...",/r/LocalLlama + /r/localLLM Recap\n1. AI Agent...,A summary of Summaries of Summaries by X.ai Gr...,"Agent ops, observability, and real-world envs\...",2025-10-23,"Agent ops, observability, and real-world envsL..."
403,https://news.smol.ai/issues/25-10-24-not-much,not much happened today,2025-10-24,10/23/2025-10/24/2025,Serving and Production Platforms: vLLM x NVIDI...,/r/LocalLlama + /r/localLLM Recap\nGLM-4.6-Air...,A summary of Summaries of Summaries by X.ai Gr...,Serving and Production Platforms: vLLM x NVIDI...,2025-10-24,Serving and Production Platforms: vLLM x NVIDI...
404,https://news.smol.ai/issues/25-10-27-minimax-m2,MiniMax M2 230BA10B — 8% of Claude Sonnet's pr...,2025-10-27,10/24/2025-10/27/2025,MiniMax M2 open-weights release: sparse MoE fo...,/r/LocalLlama + /r/localLLM Recap\n1. Open-Sou...,A summary of Summaries of Summaries by Gemini ...,MiniMax M2 open-weights release: sparse MoE fo...,2025-10-27,MiniMax M2 open-weights release: sparse MoE fo...
405,https://news.smol.ai/issues/25-10-28-openai-re...,OpenAI completes Microsoft + For-profit restru...,2025-10-28,10/27/2025-10/28/2025,"OpenAI’s new structure, Microsoft deal, and “o...",/r/LocalLlama + /r/localLLM Recap\n1. DGX Spar...,A summary of Summaries of Summaries by gpt-5\n...,"OpenAI’s new structure, Microsoft deal, and “o...",2025-10-28,"OpenAIs new structure, Microsoft deal, and ope..."


In [ ]:
# Replace @handles with incremental placeholders person_001, person_002, ...
import re
from collections import OrderedDict

handle_pattern = r'@[A-Za-z0-9_]+'  # basic handle pattern

# If handles were already replaced previously, we may have lost them. Optionally rebuild from original 'content'.
# Rebuild a fresh cleaned version (without prior handle masking) if desired.
# Comment out next two lines if you do NOT want to recompute cleaning:
# from html import unescape  # already imported earlier
# smalai['content_cleaned'] = smalai['content'].apply(clean_text)

# Gather unique handles preserving first-seen order
unique_handles = OrderedDict()
for text in smalai['content_cleaned'].fillna(''):
    for h in re.findall(handle_pattern, text):
        if h not in unique_handles:
            unique_handles[h] = None

# Build mapping
for i, h in enumerate(unique_handles.keys(), start=1):
    unique_handles[h] = f"person_{i:03d}"

print(f"Total unique handles found: {len(unique_handles)}")

# Replacement function using mapping
handle_regex = re.compile(handle_pattern)

def replace_handle(match):
    h = match.group(0)
    return unique_handles.get(h, 'person_000')  # fallback (shouldn't trigger)

smalai['content_cleaned'] = smalai['content_cleaned'].fillna('').apply(lambda txt: handle_regex.sub(replace_handle, txt))

# Verification
remaining_handles = smalai['content_cleaned'].str.contains(handle_pattern, regex=True).sum()
print(f"Rows still containing @handle pattern after replacement: {remaining_handles}")

# Show a sample of the mapping (first 15)
print("Sample mapping (first 15):")
for i, (orig, repl) in enumerate(unique_handles.items()):
    if i >= 15:
        break
    print(f"  {orig} -> {repl}")



Total unique handles found: 1501
Rows still containing @handle pattern after replacement: 0
Sample mapping (first 15):
  @fchollet -> person_001
  @_akhaliq -> person_002
  @hardmaru -> person_003
  @svpino -> person_004
  @erhartford -> person_005
  @MistralAIs -> person_006
  @CrusoeCloud -> person_007
  @WinsonDabbles -> person_008
  @abacusai -> person_009
  @arankomatsuzaki -> person_010
  @chaseleantj -> person_011
  @LangChainAI -> person_012
  @virattt -> person_013
  @llama_index -> person_014
  @AISafetyMemes -> person_015


,original_handle,placeholder
0,@fchollet,person_001
1,@_akhaliq,person_002
2,@hardmaru,person_003
3,@svpino,person_004
4,@erhartford,person_005


In [131]:
# Check results of cleaning
print("Full content of the row:\n")
for column in smalai.columns:
    print(f"{column}:")
    print(smalai.iloc[1][column])
    print("-" * 80)

Full content of the row:

url:
https://news.smol.ai/issues/24-04-02-ainews-not-much-happened-today
--------------------------------------------------------------------------------
title:
Not much happened today
--------------------------------------------------------------------------------
date:
2024-04-02 00:00:00
--------------------------------------------------------------------------------
news_period:
4/1/2024-4/2/2024
--------------------------------------------------------------------------------
twitter_recap:
all recaps done by Claude 3 Opus, best of 4 runs. We are working on clustering and flow engineering with Haiku.
AI Models and Architectures
DBRX : @DbrxMosaicAI noted DBRX is the top open-source model on the latest WildBench Leaderboard on HuggingFace , trained on 12T tokens of high-quality data with efficiency improvements like fine-grained MoE, GPT-4 tokenizer, and architecture modifications. Jamba : @AI21Labs released the Jamba whitepaper detailing the novel hybrid S

#### Remove duplicate phrases

In [ ]:
from collections import Counter
import re

def extract_phrases(text, min_words=5, max_words=15):
    """Extract phrases of varying lengths from text."""
    if pd.isna(text):
        return []
    
    words = text.split()
    phrases = []
    
    # Extract phrases of different lengths
    for n in range(min_words, min(max_words + 1, len(words) + 1)):
        for i in range(len(words) - n + 1):
            phrase = ' '.join(words[i:i+n])
            phrases.append(phrase)
    
    return phrases

# Step 1: Collect all phrases from all rows
print("Extracting phrases from all rows...")
all_phrases = []
for idx, row in smalai.iterrows():
    content = row['content_cleaned']
    if pd.notna(content):
        phrases = extract_phrases(content, min_words=5, max_words=15)
        all_phrases.extend(phrases)

print(f"Total phrases extracted: {len(all_phrases)}")

# Step 2: Count phrase frequencies
phrase_counts = Counter(all_phrases)

# Step 3: Find phrases that appear more than 10 times
duplicate_phrases = {phrase for phrase, count in phrase_counts.items() if count > 10}
print(f"Found {len(duplicate_phrases)} phrases appearing more than 10 times")

# Display top 20 most common phrases
print("\nTop 20 most common phrases:")
for phrase, count in phrase_counts.most_common(20):
    print(f"  {count}x: {phrase[:100]}...")  # Show first 100 chars

Extracting phrases from all rows...
Total phrases extracted: 17113272
Total phrases extracted: 17113272
Found 1022 phrases appearing more than 10 times

Top 20 most common phrases:
Found 1022 phrases appearing more than 10 times

Top 20 most common phrases:
  299x: summary of Summaries of Summaries...
  297x: of Summaries of Summaries by...
  297x: summary of Summaries of Summaries by...
  211x: all recaps done by Claude...
  211x: /r/Oobabooga, /r/MachineLearning, /r/OpenAI, /r/ClaudeAI, /r/StableDiffusion,...
  210x: AI Subreddit Recap/r/Singularity, /r/Oobabooga, /r/MachineLearning,...
  210x: Subreddit Recap/r/Singularity, /r/Oobabooga, /r/MachineLearning, /r/OpenAI,...
  210x: Recap/r/Singularity, /r/Oobabooga, /r/MachineLearning, /r/OpenAI, /r/ClaudeAI,...
  210x: AI Subreddit Recap/r/Singularity, /r/Oobabooga, /r/MachineLearning, /r/OpenAI,...
  210x: Subreddit Recap/r/Singularity, /r/Oobabooga, /r/MachineLearning, /r/OpenAI, /r/ClaudeAI,...
  210x: Recap/r/Singularity, /r/Oobab

In [134]:
def remove_duplicate_phrases(text, phrases_to_remove):
    """Remove duplicate phrases from text."""
    if pd.isna(text):
        return text
    
    # Sort phrases by length (longest first) to avoid partial replacements
    sorted_phrases = sorted(phrases_to_remove, key=len, reverse=True)
    
    result = text
    for phrase in sorted_phrases:
        # Use regex to remove the phrase (case-insensitive, word boundaries)
        pattern = re.escape(phrase)
        result = re.sub(pattern, '', result, flags=re.IGNORECASE)
    
    # Clean up extra whitespace
    result = re.sub(r'\s+', ' ', result).strip()
    
    return result if result else pd.NA

# Step 4: Remove duplicate phrases from content_cleaned
print("Removing duplicate phrases from content...")
smalai['content_deduped'] = smalai['content_cleaned'].apply(
    lambda x: remove_duplicate_phrases(x, duplicate_phrases)
)

print("\nDeduplication complete!")
print(f"Original content length (first row): {len(str(smalai.iloc[0]['content_cleaned'])) if pd.notna(smalai.iloc[0]['content_cleaned']) else 0}")
print(f"Deduped content length (first row): {len(str(smalai.iloc[0]['content_deduped'])) if pd.notna(smalai.iloc[0]['content_deduped']) else 0}")
print("Number texts: ", len(smalai))

smalai

Removing duplicate phrases from content...

Deduplication complete!
Original content length (first row): 8133
Deduped content length (first row): 7833
Number texts:  406


,url,title,date,news_period,twitter_recap,reddit_recap,discord_recap,content,date_extracted,content_cleaned,content_deduped
0,https://news.smol.ai/issues/24-04-01-ainews-ad...,AdamW -> AaronD?,2024-04-01,3/28/2024-4/1/2024,"all recaps done by Claude 3 Opus, best of 4 ru...","Across r/LocalLlama, r/machinelearning, r/open...",None,"all recaps done by Claude 3 Opus, best of 4 ru...",2024-04-01,"all recaps done by Claude 3 Opus, best of 4 ru...",Capabilities and LimitationsLimitations of cur...
1,https://news.smol.ai/issues/24-04-02-ainews-no...,Not much happened today,2024-04-02,4/1/2024-4/2/2024,"all recaps done by Claude 3 Opus, best of 4 ru...","Across r/LocalLlama, r/machinelearning, r/open...",A summary of Summaries of Summaries\nClaude 3 ...,"all recaps done by Claude 3 Opus, best of 4 ru...",2024-04-02,"all recaps done by Claude 3 Opus, best of 4 ru...",Models and ArchitecturesDBRX : person_019 note...
2,https://news.smol.ai/issues/24-04-03-ainews-re...,ReALM: Reference Resolution As Language Modeling,2024-04-03,4/2/2024-4/3/2024,"all recaps done by Claude 3 Opus, best of 4 ru...","Across r/LocalLlama, r/machinelearning, r/open...",A summary of Summaries of Summaries\nAdvanceme...,"all recaps done by Claude 3 Opus, best of 4 ru...",2024-04-03,"all recaps done by Claude 3 Opus, best of 4 ru...",all recaps done by with Haiku.Anthropic Resear...
3,https://news.smol.ai/issues/24-04-04-ainews-co...,"Cohere Command R+, Anthropic Claude Tool Use, ...",2024-04-04,4/3/2024-4/4/2024,"all recaps done by Claude 3 Opus, best of 4 ru...","Across r/LocalLlama, r/machinelearning, r/open...",A summary of Summaries of Summaries\nLLM Advan...,"all recaps done by Claude 3 Opus, best of 4 ru...",2024-04-04,"all recaps done by Claude 3 Opus, best of 4 ru...",all recaps done by with Haiku.Cohere Command R...
4,https://news.smol.ai/issues/24-04-05-ainews-mi...,Mixture of Depths: Dynamically allocating comp...,2024-04-05,4/4/2024-4/5/2024,"all recaps done by Claude 3 Opus, best of 4 ru...","Across r/LocalLlama, r/machinelearning, r/open...",A summary of Summaries of Summaries\n1. Cuttin...,"all recaps done by Claude 3 Opus, best of 4 ru...",2024-04-05,"all recaps done by Claude 3 Opus, best of 4 ru...",Models and ArchitecturesGoogles Training LLMs ...
...,...,...,...,...,...,...,...,...,...,...,...
402,https://news.smol.ai/issues/25-10-23-not-much,not much happened today,2025-10-23,10/22/2025-10/23/2025,"Agent ops, observability, and real-world envs\...",/r/LocalLlama + /r/localLLM Recap\n1. AI Agent...,A summary of Summaries of Summaries by X.ai Gr...,"Agent ops, observability, and real-world envs\...",2025-10-23,"Agent ops, observability, and real-world envsL...","Agent ops, observability, and real-world envsL..."
403,https://news.smol.ai/issues/25-10-24-not-much,not much happened today,2025-10-24,10/23/2025-10/24/2025,Serving and Production Platforms: vLLM x NVIDI...,/r/LocalLlama + /r/localLLM Recap\nGLM-4.6-Air...,A summary of Summaries of Summaries by X.ai Gr...,Serving and Production Platforms: vLLM x NVIDI...,2025-10-24,Serving and Production Platforms: vLLM x NVIDI...,Serving and Production Platforms: vLLM x NVIDI...
404,https://news.smol.ai/issues/25-10-27-minimax-m2,MiniMax M2 230BA10B — 8% of Claude Sonnet's pr...,2025-10-27,10/24/2025-10/27/2025,MiniMax M2 open-weights release: sparse MoE fo...,/r/LocalLlama + /r/localLLM Recap\n1. Open-Sou...,A summary of Summaries of Summaries by Gemini ...,MiniMax M2 open-weights release: sparse MoE fo...,2025-10-27,MiniMax M2 open-weights release: sparse MoE fo...,MiniMax M2 open-weights release: sparse MoE fo...
405,https://news.smol.ai/issues/25-10-28-openai-re...,OpenAI completes Microsoft + For-profit restru...,2025-10-28,10/27/2025-10/28/2025,"OpenAI’s new structure, Microsoft deal, and “o...",/r/LocalLlama + /r/localLLM Recap\n1. DGX Spar...,A summary of Summaries of Summaries by gpt-5\n...,"OpenAI’s new structure, Microsoft deal, and “o...",2025-10-28,"OpenAIs new structure, Microsoft deal

In [135]:
# Compare original vs deduped content for first row
print("="*80)
print("ORIGINAL CONTENT (first 1000 chars):")
print("="*80)
print(str(smalai.iloc[0]['content_cleaned'])[:1000])
print("\n" + "="*80)
print("DEDUPED CONTENT (first 1000 chars):")
print("="*80)
print(str(smalai.iloc[0]['content_deduped'])[:1000])


ORIGINAL CONTENT (first 1000 chars):
all recaps done by Claude 3 Opus, best of 4 runs. We are working on clustering and flow engineering with Haiku.AI Capabilities and LimitationsLimitations of current AI systems : person_001 noted that memorization, which ML has solely focused on, is not intelligence. Any task that does not involve significant novelty and uncertainty can be solved via memorization, but skill is never a sign of intelligence. person_001 shared a paper introducing a formal definition of intelligence and benchmark, noting that current state-of-the-art LLMs like Gemini Ultra, Claude 3, or GPT-4 are not able to score higher than a few percents on that benchmark. Limitations of benchmarks in assessing AI capabilities : person_002 questioned if we are on the right way for evaluating large vision-language models (LVLMs). They identified two primary issues in current benchmarks: visual content being unnecessary for many samples and unintentional data leakage in training . Poten

In [136]:
smalai_1 = smalai[['title', 'date_extracted','content_deduped']].copy()

# Add token count column
def num_tokens_from_string(string, encoding_name):
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string if isinstance(string, str) else '', disallowed_special=()))
    return num_tokens

smalai_1['token_count'] = smalai_1['content_deduped'].apply(lambda text: num_tokens_from_string(text, "cl100k_base"))
smalai_1 = smalai_1[smalai_1['token_count']>0]

print("Total tokens:", smalai_1['token_count'].sum())
print("Min tokens:", smalai_1['token_count'].min())
print("Max tokens:", smalai_1['token_count'].max())
smalai_1


Total tokens: 2171701
Min tokens: 812
Max tokens: 15062


,title,date_extracted,content_deduped,token_count
0,AdamW -> AaronD?,2024-04-01,Capabilities and LimitationsLimitations of cur...,1666
1,Not much happened today,2024-04-02,Models and ArchitecturesDBRX : person_019 note...,2989
2,ReALM: Reference Resolution As Language Modeling,2024-04-03,all recaps done by with Haiku.Anthropic Resear...,1897
3,"Cohere Command R+, Anthropic Claude Tool Use, ...",2024-04-04,all recaps done by with Haiku.Cohere Command R...,1908
4,Mixture of Depths: Dynamically allocating comp...,2024-04-05,Models and ArchitecturesGoogles Training LLMs ...,2233
...,...,...,...,...
402,not much happened today,2025-10-23,"Agent ops, observability, and real-world envsL...",4370
403,not much happened today,2025-10-24,Serving and Production Platforms: vLLM x NVIDI...,4874
404,MiniMax M2 230BA10B — 8% of Claude Sonnet's pr...,2025-10-27,MiniMax M2 open-weights release: sparse MoE fo...,5146
405,OpenAI completes Microsoft + For-profit restru...,2025-10-28,"OpenAIs new structure, Microsoft deal, and ope...",5946


### Save cleaned dataset

In [137]:
file_path = r'C:\Users\Denis_Davydov2\OneDrive - EPAM\Prophet_AI_docs\Datasets\AI_skills'
file_name = f'smolai_texts_{date.today()}.csv'

smalai_1.to_csv(file_path + '\\' + file_name, index=False, encoding='utf-8-sig')
print(f"File {file_name} \nuploaded to:", file_path)
print("Total rows", len(smalai_1))

File smolai_texts_2025-10-31.csv 
uploaded to: C:\Users\Denis_Davydov2\OneDrive - EPAM\Prophet_AI_docs\Datasets\AI_skills
Total rows 403
